In [ ]:
# в этом ноутбуке я считаю профиты для лонг трейдов со скользящим стоплоссом.
# собираюсь использовать это как раметку данных.
import pandas as pd
import numpy as np
import pandas_ta as ta
import forecast
from quote_chart import create_chart_app
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime

fee_percent = 0.07

trailing_stop_loss_percent = 0.15

In [ ]:
data = forecast.get_df('BTCUSD', '', '1T')

In [ ]:
def get_volatility():
    df = data
    closes = df['close']
    return np.abs(np.log(closes/closes.shift(1)))
volatility = get_volatility()
data['volatility'] = volatility
data['volatility_ema'] = ta.ema(volatility, 20)
data['volatility_ema1k'] = ta.ema(volatility, 10*1000)
data['volatility_multiplier'] = (1+data['volatility_ema']/data['volatility_ema1k']) **1.5

In [ ]:
def get_trailing_stop_loss_long_profits():
    df = data
    opens = df['open'].to_numpy()
    highs = df['high'].to_numpy()
    lows = df['low'].to_numpy()
    closes = df['close'].to_numpy()
    volatility = df['volatility_multiplier']
    # volatility.fillna(1, inplace=True)
    volatility = volatility.to_numpy()

    n = len(opens)
    i = 0
    profits = []
    stop_loss_sizes = []
    stop_loss_prices = []
    maximums = []
    while i < n:
        #print(i)
        max = highs[i]
        max_i = i
        sl_size = max * 0.01 * trailing_stop_loss_percent * volatility[i]
        sl_price = max - sl_size
        # starting from the current position look forward for trailing stop loss closing.
        j = i
        j0 = i
        local_stop_loss_sizes = []
        local_stop_loss_prices = []
        local_maximums = []
        while j < n:
            # if price went lower than current stop loss then assume that the position was closed at the stop loss price.
            if lows[j] < sl_price or j == n - 1:
                profits.append(sl_price - opens[i])
                # all openings between the current opening and the max_i will be closed on the same stop loss position.
                while i < max_i:
                    i += 1
                    profits.append(sl_price - opens[i])
                    stop_loss_sizes.append(local_stop_loss_sizes[i - j0])
                    stop_loss_prices.append(local_stop_loss_prices[i - j0])
                    maximums.append(local_maximums[i - j0])
                break # break from while j loop.
            # if there is a new max then save it, it's position and recalculate trailing stop loss.
            if max < highs[j]:
                max = highs[j]
                max_i = j
                sl_size = max * 0.01 * trailing_stop_loss_percent * volatility[j]
                if sl_price < max - sl_size:
                    sl_price = max - sl_size
            local_stop_loss_sizes.append(sl_size)
            local_stop_loss_prices.append(sl_price)
            local_maximums.append(max)
            j += 1
        stop_loss_sizes.append(sl_size)
        stop_loss_prices.append(sl_price)
        maximums.append(max)
        i += 1
    return (profits, stop_loss_sizes, stop_loss_prices, maximums)

(profits, stop_loss_sizes, stop_loss_prices, maximums) = get_trailing_stop_loss_long_profits()
data['profit'] = profits
data['stop_loss_size'] = stop_loss_sizes
data['stop_loss_price'] = stop_loss_prices
data['maximum'] = maximums
# print(len(df))
# print(len(profits))

In [ ]:
resampled_df = data.copy()

# called when a period button is pressed under the plot.
def on_period_change(button_id):
    global resampled_df, selected_period
    if button_id == '':
        return
    selected_period = button_id
    resampled_df = data.resample(selected_period).agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum',
        'volatility_multiplier': 'last',
        'profit': 'first',
        'stop_loss_size': 'last',
        'stop_loss_price': 'last',
        'maximum': 'last',
    })
    resampled_df = resampled_df.dropna()

def create_figure(x0, x1):
    if x0 is not None:
        print(x0)
        df = resampled_df[x0:x1]
    else:
        #df = resampled_df[-100:]#
        xx0 = datetime.strptime('2024-05-20 00:03:44.250000', '%Y-%m-%d %H:%M:%S.%f')
        xx1 = datetime.strptime('2024-05-21 02:03:44.250000', '%Y-%m-%d %H:%M:%S.%f')
        df = resampled_df[xx0:xx1]
        
    # define multiple panes. The top pane will be for the main price chart with candles. The second pane is for volumes.
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, 
                vertical_spacing=0.01,
                row_heights=[0.8, 0.2, 0.2],
                specs=[[{"secondary_y": True}], [{"secondary_y": True}], [{"secondary_y": True}]])
    # plot the main chart with price candles.
    fig.add_trace(go.Candlestick(x=df.index,
                                 open=df['open'],
                                 high=df['high'],
                                 low=df['low'],
                                 close=df['close'],
                                 name='ohlc'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['open'] + df['profit'], mode='lines', line=dict(color='black'), name='open+profit'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['open'] + df['profit'] - (df['maximum'] - df['close']), mode='lines', line=dict(color='orange'), name='discounted profit'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['open'] * (1 + 0.01*fee_percent*2), mode='lines', line=dict(color='blue'), name='fee'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['stop_loss_price'], mode='lines', line=dict(color='red'), name='stop_loss_pirce'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['maximum'], mode='lines', line=dict(color='green'), name='maximum'), row=1, col=1)
    # fig.add_trace(go.Scatter(x=df.index, y=df['open'] * (1 - trailing_stop_loss_percent * 0.01), mode='lines', line=dict(color='blue'), name='open-tsl'), row=1, col=1)

    fig.add_trace(go.Bar(x=df.index, y=df['volume'], name='Volume'), row=2, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['close'], mode='lines', line=dict(color='black'), name='close'), row=2, col=1, secondary_y=True)

    # fig.add_trace(go.Scatter(x=df.index, y=df['profit'], mode='lines', line=dict(color='blue'), name='profit'), row=3, col=1)

    # fig.add_trace(go.Scatter(x=df.index, y=df['volatility'], mode='lines', line=dict(color='blue'), name='volatility'), row=3, col=1)
    # fig.add_trace(go.Scatter(x=df.index, y=df['volatility_ema'], mode='lines', line=dict(color='red'), name='volatility_ema'), row=3, col=1)
    # fig.add_trace(go.Scatter(x=df.index, y=df['volatility_ema1k'], mode='lines', line=dict(color='green'), name='volatility_ema1k'), row=3, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=1+df['volatility_multiplier'], mode='lines', line=dict(color='green'), name='volatility_multiplier'), row=3, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=0 * df['volatility_multiplier'], mode='lines', line=dict(color='gray'), name='zero'), row=3, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['stop_loss_size'], mode='lines', line=dict(color='black'), name='stop_loss_size'), row=3, col=1, secondary_y=True)
    fig.add_trace(go.Scatter(x=df.index, y=df['open'] * 0.01 * fee_percent * 2, mode='lines', line=dict(color='blue'), name='fee'), row=3, col=1, secondary_y=True)
    fig.add_trace(go.Scatter(x=df.index, y=df['open'] * 0, mode='lines', line=dict(color='gray'), name='zero'), row=3, col=1, secondary_y=True)

    # mpf.make_addplot(df['open'] + df['profits'], panel=0, color='black'),
    # mpf.make_addplot(df['open'] * (1 - trailing_stop_loss_percent * 0.01), panel=0, color='blue'),
    # mpf.make_addplot(df['profits'], panel=2, color='black'),

    # set the default dragmode to pan, remove the range slider because i use zoom/pan instead of it.
    fig.update_layout(
        dragmode='pan',
        xaxis_rangeslider_visible=False,
        width=1200, # px
        height=600,
        margin=dict(l=50, r=0, t=0, b=0),
        yaxis=dict(side='right'),
        yaxis2=dict(side='left'),
        yaxis3=dict(side='right'),
        yaxis4=dict(side='left'),
        yaxis5=dict(side='right'),
        yaxis6=dict(side='left'),
        )
    fig.update_xaxes(
        ticklabelposition="outside right",  # keep labels on the right so that they don't affect margin-left.
    )
    return fig

app = create_chart_app(create_figure, on_period_change, period_buttons=None, debug=False)
app.run_server(debug=True)